In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import random
from scipy.special import iv

# Some Conversion functions 

In [2]:
def dBm_to_watts(dBm):
    # Converts dBm (decibels relative to one milliwatt) to Watts
    return 10 ** (dBm / 10) / 1000
    
def watts_to_dBm(watts):
    # Converts Watts to dBm (decibels relative to one milliwatt)
    return 10 * math.log10(1000 * watts)

def dB_to_linear_scale(dB):
    # Converts dB (decibels) to linear scale
    return 10 ** (dB / 10)

def linear_scale_to_dB(linear_scale):
    # Converts linear scale to dB (decibels).
    return 10 * math.log10(linear_scale)
    
def convert_mega(value):
    # Converts from mega
    return 10 ** 6 * value

def convert_giga(value):
    # Converts from giga
    return 10 ** 9 * value

![FORMULA](pictures/formula.png) 

# Quantities declaration

In [3]:
c = convert_giga(0.3)
carrier_frequency = convert_giga(30)
wavelength = c / carrier_frequency
number_of_flows = 80
number_of_time_slots = 2000
slot_duration = 0.1
fading_depth = 2
background_noise = dBm_to_watts(-134) / convert_mega(1)
system_bandwidth = convert_mega(2000)
transmission_power = dBm_to_watts(40)
average_power_of_uav = dBm_to_watts(30)
peak_power_of_uav = 2 * average_power_of_uav
transceiver_efficiency = 0.8
average_velocity_vehicle = 28
height_of_uav = 100
speed_of_uav = 20
radius_of_uav = 5
transmit_power_uav = dBm_to_watts(30)
coverage_radius = 500
pl_factor_for_v2v = 2.5
pl_factor_for_u2v = 2
rician_k_factor = dB_to_linear_scale(9)
interference_threshold = 10 ** -3
si_cancellation_level = 10 ** -9
maximum_antenna_gain = dB_to_linear_scale(20)
half_power_beamwidth = 30
total_power = 1
number_of_uavs = 3
distance_u2v = np.zeros((number_of_uavs + 1, number_of_flows + 1))
distance_v2u = np.zeros((number_of_flows + 1, number_of_uavs + 1))
small_scale_power_fading_matrix_u2v = np.zeros((number_of_uavs + 1, number_of_flows + 1))
small_scale_power_fading_matrix_v2u = np.zeros((number_of_flows + 1, number_of_uavs + 1))
channel_power_gain_matrix = np.zeros((number_of_flows + 1, number_of_flows + 1))
received_signal_power_matrix = np.zeros((number_of_flows + 1, number_of_flows + 1))
distance_matrix = np.zeros((number_of_flows + 1, number_of_flows + 1))
number_of_slots_for_each_flow = np.zeros(number_of_flows + 1)
# 2-D Binary Matrices
a_v2v = np.ones((number_of_flows + 1, number_of_flows + 1))
b_v2v = np.ones((number_of_flows + 1, number_of_flows + 1))
a_u2v = np.ones((number_of_flows + 1, number_of_flows + 1))
b_u2v = np.ones((number_of_flows + 1, number_of_flows + 1))
# Sinr and datarates

sinr_v2u = np.zeros((number_of_flows + 1, number_of_uavs + 1))
sinr_u2v = np.zeros((number_of_uavs + 1, number_of_flows + 1))

datarate_v2u = np.zeros((number_of_flows + 1, number_of_uavs + 1))
datarate_u2v = np.zeros((number_of_uavs + 1, number_of_flows + 1))
datarate_without_interference = np.zeros(number_of_flows + 1)

In [4]:
def generate_exponential_random_variable(rate):
    """Generate a random variable following an exponential distribution with given rate."""
    return -math.log(1.0 - random.random()) / rate

In [18]:
def simulate_vehicle_arrivals(v = average_velocity_vehicle, simulation_time = (number_of_time_slots * slot_duration), max_vehicles_per_lane=60):
    
    # lambda_rate = v * 2  # Arrival rate (vehicles per second)
    lambda_rate = 0.1 * 2
    
    arrival_times = []
    positions = []
    current_times = [0, 0, 0]  # Track current time for each lane
    current_positions = [0, 0, 0]  # Track current position for each lane
    vehicle_counts = [0, 0, 0]  # Track the number of vehicles in each lane
    
    while min(current_times) < simulation_time and sum(vehicle_counts) < max_vehicles_per_lane * 3:
        # Generate next inter-arrival time
        inter_arrival_time = generate_exponential_random_variable(0.2)
        
        # Choose a random lane for the current vehicle
        lane = random.randint(1, 3)
        lane_index = lane - 1
        
        # Check if the chosen lane has reached the maximum number of vehicles
        if vehicle_counts[lane_index] >= max_vehicles_per_lane:
            continue
        
        current_times[lane_index] += inter_arrival_time
        
        if current_times[lane_index] > simulation_time:
            break
        
        # Calculate safe distance for the current vehicle
        d_n = generate_exponential_random_variable(lambda_rate)
        safe_distance = max(2, d_n)
        
        # Update position in the chosen lane
        current_positions[lane_index] += safe_distance
        
        # Record arrival time, lane, and position
        arrival_times.append(current_times[lane_index])
        positions.append((lane, current_positions[lane_index]))
        vehicle_counts[lane_index] += 1
    
    return list(zip(arrival_times, positions))


In [4]:
#Random Throughputs for each flow between [0.1,1] GBPS
throughputs = convert_giga(np.random.uniform(0.1, 1, size = number_of_flows + 1))

In [5]:
def euclidean_distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

In [6]:
# Calculating segmentation point of line using section formula
def section_formula(x1, y1, x2, y2, m, n):
    Px = (m * x2 + n * x1) / (m + n)
    Py = (m * y2 + n * y1) / (m + n)
    return Px, Py

In [7]:
def calculate_antenna_gain(direction): #To be implemented. Angle is ambiguous
    # deduction=min((direction/half_power_beamwidth)**2,26)
    # gain=maximum_antenna_gain-deduction
    return maximum_antenna_gain

In [8]:
# Generating channel power gain distribution
def calculate_channel_power_gain_distribution():
    return np.random.gamma(shape = fading_depth, scale = 1 / fading_depth)

#     for i in range(1, number_of_flows + 1):
#         for j in range(1, number_of_flows + 1):
#             channel_power_gain_matrix[i, j] = np.random.gamma(shape = fading_depth, scale = 1 / fading_depth)

In [9]:
# Calculating received signal power from direct link
def calculate_received_signal_power_direct_link(distance):
    kv = (wavelength / (4 * math.pi)) ** pl_factor_for_v2v
    signal_power = kv * transmission_power * maximum_antenna_gain * calculate_channel_power_gain_distribution() * (distance ** (-pl_factor_for_v2v))
    return signal_power

In [10]:
# Mutual interference between 2 flows
def calculate_mutual_interference(distance):
    kv = (wavelength / (4 * math.pi)) ** pl_factor_for_v2v
    signal_power = kv * transmission_power * calculate_antenna_gain(0) * calculate_channel_power_gain_distribution() * (distance ** (-pl_factor_for_v2v))
    return signal_power

In [11]:
# Storing values of received signal power for each flow in a 2D matrix
def calculate_received_signal_power():
    for i in range(1, number_of_flows + 1):
        for j in range(1, number_of_flows + 1):
            if i == j:
                received_signal_power_matrix[i, i] = calculate_received_signal_power_direct_link(i)
            else:
                received_signal_power_matrix[j, i] = calculate_mutual_interference(j, i)  

In [12]:
# SI cancellation level calculation
def calculate_rsi():
    return si_cancellation_level * transmission_power

In [13]:
# SINR and Datarate calculation for v2v links
def calculate_sinr_and_datarate_v2v(group):
    number_of_flows_in_group = len(group)
    sinr_v2v = np.zeros(number_of_flows_in_group + 1)
    datarate_v2v = np.zeros(number_of_flows_in_group + 1)
    rsi = calculate_rsi()
    for i in range(1, number_of_flows_in_group + 1):
        flow_index_i = group[i]
        num = received_signal_power_matrix[flow_index_i, flow_index_i]
        denom = background_noise * system_bandwidth
        for j in range(1, number_of_flows_in_group + 1):
            if j == i:
                continue
            flow_index_j = group[j]
            denom += (a_v2v[flow_index_j, flow_index_i] * received_signal_power_matrix[flow_index_j, flow_index_i]) + (b_v2v[flow_index_j, flow_index_i] * rsi)
        sinr_v2v[i] = num / denom
        datarate_v2v[i] = transceiver_efficiency * system_bandwidth * (math.log1p(sinr_v2v[i]) / math.log(2))
        return sinr_v2v , datarate_v2v

In [14]:
# Function for generating non central chi square distribution 
def calculate_non_central_chi_square_distribution(small_scale_power_fading, total_power):
    ans = (rician_k_factor + 1) * np.exp(-rician_k_factor) * np.exp(-(rician_k_factor + 1) * small_scale_power_fading / total_power)
    ans *= iv(0, 2 * np.sqrt(rician_k_factor * (rician_k_factor + 1) * small_scale_power_fading / total_power)) / total_power
    return ans

In [15]:
# Calculating small scale power fading using non central chi square distribution
def calculate_small_scale_power_fading():
    small_scale_power_fading = np.random.uniform(0,0.5)
    return calculate_non_central_chi_square_distribution(small_scale_power_fading, total_power)

In [16]:
# SINR and Datarate calculation for u2v links
def calculate_sinr_and_datarate_u2v(group,uavs_in_group):
    number_of_flows_in_group = len(group)
    number_of_uavs_in_group = len(uavs_in_group)
    sinr_u2v = np.zeros((number_of_uavs_in_group + 1, number_of_flows_in_group + 1))
    datarate_u2v = np.zeros((number_of_uavs_in_group + 1, number_of_flows_in_group + 1))
    rsi = calculate_rsi()
    ku = (wavelength / (4 * math.pi)) ** pl_factor_for_u2v
    for u in range(1, number_of_uavs_in_group + 1):
        uav_u = uavs_in_group[u]
        for k in range(1, number_of_flows_in_group + 1):
            flow_k = group[k]
            num = ku * transmit_power_uav * maximum_antenna_gain * (distance_u2v[uav_u, flow_k] ** (-pl_factor_for_u2v)) * small_scale_power_fading_matrix_u2v[uav_u, flow_k]
            denom = background_noise * system_bandwidth
            for w in range(1, number_of_flows_in_group + 1):
                if w == k:
                    continue
                flow_w = group[w]
                denom += (a_u2v[flow_w, flow_k] * received_signal_power_matrix[flow_w, flow_k]) + (b_u2v[flow_w, flow_k] * rsi)
            sinr_u2v[u, k] = num / denom
            datarate_u2v[u, k] = transceiver_efficiency * system_bandwidth * (math.log1p(sinr_u2v[u, k]) / math.log(2))
    return sinr_u2v,datarate_u2v

In [17]:
# SINR and Datarate calculation for v2u links
def calculate_sinr_and_datarate_v2u(group,uavs_in_group):
    number_of_flows_in_group = len(group)
    number_of_uavs_in_group = len(uavs_in_group)
    sinr_v2u = np.zeros((number_of_flows_in_group + 1, number_of_uavs_in_group + 1))
    datarate_v2u = np.zeros((number_of_flows_in_group + 1, number_of_uavs_in_group + 1))
    kv = (wavelength / (4 * math.pi)) ** pl_factor_for_v2v
    for k in range(1, number_of_flows_in_group + 1):
        flow_k = group[k]
        for u in range(1, number_of_uavs_in_group + 1):
            uav_u = uavs_in_group[u]
            num = kv * transmission_power * maximum_antenna_gain * (distance_v2u[flow_k, uav_u] ** (-pl_factor_for_v2v)) * small_scale_power_fading_matrix_v2u[flow_k, uav_u]
            denom = background_noise * system_bandwidth
            sinr_v2u[k, u] = num / denom
            datarate_v2u[k, u] = transceiver_efficiency * system_bandwidth * (math.log1p(sinr_v2u[k, u]) / math.log(2))

In [18]:
def calculate_sinr(num, denom):
    return num / denom

In [19]:
def calculate_data_rate(num, denom):
    return int(transceiver_efficiency * system_bandwidth * (math.log1p(calculate_sinr(num, denom)) / math.log(2)))

# RI (Relative Interference)

![RI1](pictures/ri1.png) 

In [20]:
def calculate_relative_interference1():
    num = background_noise * system_bandwidth + calculate_rsi()
    denom = transmission_power
    return num / denom

![RI2](pictures/ri2.png) 

In [21]:
def calculate_relative_interference2(sender, receiver):
    num = background_noise * system_bandwidth + calculate_mutual_interference(euclidean_distance(sender.calculate_position(0), receiver.calculate_position(0)))
    denom = transmission_power
    return num / denom

# Data Rate without interference

In [22]:
# Calculating by eliminating MUI and SI from denominator and using Shannon's capacity formula
def calculate_datarate_without_interference(flows):
    for i in range(1, number_of_flows + 1):
        sender = flows[i][0]
        receiver = flows[i][1]
        snr = calculate_received_signal_power_direct_link(euclidean_distance(*sender.calculate_position(0), *receiver.calculate_position(0))) / (background_noise * system_bandwidth)
        datarate_without_interference[i] = transceiver_efficiency * system_bandwidth * (math.log1p(snr) / math.log(2))

In [19]:
tempo = simulate_vehicle_arrivals()

In [20]:
tempo

[(0.9550880603824028, (1, 2)),
 (3.4709014975443493, (1, 4)),
 (13.830379336406317, (1, 6)),
 (23.486958448154944, (1, 8)),
 (1.9357077244232597, (2, 2)),
 (3.0322208014795935, (2, 4)),
 (15.717293993450076, (3, 2)),
 (35.92909894852134, (3, 4)),
 (42.53230823974782, (3, 6)),
 (55.98655145465345, (3, 8)),
 (24.9065977618596, (1, 10)),
 (29.922808015933697, (1, 12)),
 (67.35296597583957, (3, 10)),
 (32.619536889686316, (1, 14)),
 (9.09811243012355, (2, 6)),
 (67.57634958103765, (3, 12)),
 (15.395338442342211, (2, 8)),
 (22.497303273294285, (2, 10)),
 (32.84873952196376, (1, 16)),
 (25.188505680058057, (2, 12)),
 (32.210761821615975, (2, 14)),
 (32.54111821583457, (2, 16)),
 (67.63574710856751, (3, 14)),
 (72.01787573434541, (3, 16)),
 (72.24097362721771, (3, 18)),
 (36.692118533913174, (2, 18)),
 (73.14293050286467, (3, 20)),
 (47.44829452527623, (2, 20)),
 (75.94321156471463, (3, 22)),
 (52.26402550033077, (2, 22)),
 (81.00025946160612, (3, 24)),
 (36.83125658780358, (1, 18)),
 (39.376

In [21]:
len(tempo)

96